In [ ]:
from pynq.overlays.base import BaseOverlay
base = BaseOverlay('base.bit')

In [ ]:
base.init_rf_clks()

In [ ]:
#!/usr/bin/env python
"""
"""
import redis
import re
import numpy as np
import json
import time
    
    
board_name = "bu_rfsoc"
max_len    = 10000000
r = redis.Redis(host='155.41.101.109', port=6379, db=0)


def set_transmitter_channel(channel, enable, gain, frequency):
    channel.control.enable = enable
    channel.control.gain = gain
    channel.dac_block.MixerSettings['Freq'] = frequency
    
    

def board_requests_handler(msg):
    board_channel=1
    if board_channel == 1:
        set_transmitter_channel(base.radio.transmitter.channel[1], True, 0.8,  750)
        
    number_samples=32768
    print(f"GOT REQUEST: {msg}")
    r.srem("active_command_boards", board_name)
    channel = msg['channel'].decode()
    # get id for request from channel name
    req_id = re.findall(r'\d+', channel)[0] 
    r.delete(f"board-responses:{board_name}:{req_id}:metadata")
    r.delete(f"board-responses:{board_name}:{req_id}:real")
    r.delete(f"board-responses:{board_name}:{req_id}:imag")
    r.delete(f"board-responses:{board_name}:{req_id}:complete")
    req_params = json.loads(msg['data'])
    
    # duration
    duration = req_params['duration']
    
    fc=base.radio.transmitter.channel[board_channel].dac_block.MixerSettings['Freq']*1e6
    fs=base.radio.transmitter.channel[board_channel].dac_block.BlockStatus['SamplingFreq']*1e9
    
    metadata = {
        'sfreq': fs,
        'cfreq': fc,
        'number_samples': number_samples,
    }
    
    r.set(f"board-responses:{board_name}:{req_id}:complete", 'False')
    
    print(f"sending metadata: {metadata}")
    r.xadd(f"board-responses:{board_name}:{req_id}:metadata", {'data': json.dumps(metadata)})
    
    
    start = time.time()
    
    end   = start + duration
    
    cdata =[]
    count = 0
    
    b = base.radio.receiver.channel[board_channel]
    try:
        while time.time() < end:
            cdata.append(b.transfer(number_samples))
            count += 1
            print(f"count: {count}")
        for d in cdata:
            xreal=d.real
            xreal_json = json.dumps(xreal.tolist())
            r.xadd(f"board-responses:{board_name}:{req_id}:real", {'data': xreal_json}, maxlen=max_len)
            ximag=d.imag
            ximag_json = json.dumps(ximag.tolist())
            r.xadd(f"board-responses:{board_name}:{req_id}:imag", {'data': ximag_json}, maxlen=max_len)
    finally:
        print("ENTER BOARD_REQUEST_HANDLER FINALLY")
    
    r.set(f"board-responses:{board_name}:{req_id}:complete", 'True')
    r.sadd("active_command_boards", board_name)
    
        

    
def run_stream():
    """Set ups a redis connection and updates data to the stream."""
    p = r.pubsub(ignore_subscribe_messages=True)
    
    
    p.psubscribe(**{f'board-requests:{board_name}:*': board_requests_handler})
    
    r.set(f'board-request-id:{board_name}', 0)
    r.sadd("active_command_boards", board_name)
    
    print("Waiting...")
    try:
        while True:
            p.get_message()
            time.sleep(.01)
    finally:
        print("ENTER FINALLY")
        r.srem("active_command_boards", board_name)
    
        
if __name__ == '__main__':
    run_stream()
